In [ ]:
import numpy as np
from numpy import random
import scipy
#import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, pearsonr

In [ ]:
a = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
for i in range(10):
    print(random.choice(a,len(a)))

In [ ]:
N = np.linspace(1,40,100)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(N, 1-(1-1/N)**N,'k',linewidth=2)
plt.hlines(1-1/np.e,-1,40,'m',linestyle='--')
plt.xlim(0.5,40)
plt.ylim(0.5,1.0)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('N',fontsize=22)
plt.grid(linestyle='--')

# Bootstrap example

In [ ]:
A = multivariate_normal( mean=[0,0], cov=[[1.0, 0.7], [0.7, 1.0]] )

In [ ]:
sample = A.rvs(size=100)

In [ ]:
x = sample[:,0]
y = sample[:,1]

lim = 3
xgrid, ygrid = np.mgrid[-lim:lim:.01, -lim:lim:.01]
pos = np.dstack((xgrid, ygrid))
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.contourf(xgrid, ygrid, A.pdf(pos))
ax2.scatter(x=x,y=y,color='magenta',marker='.')

In [ ]:
rsamp,_ = pearsonr(sample[:,0],sample[:,1])

In [ ]:
rsamp

# Do this many times

In [ ]:
rsamp = np.empty(1000)

for i in range(1000):
    sample = A.rvs(size=100)
    rsamp[i],_ = pearsonr(sample[:,0],sample[:,1])
    
print("%.4f +/- %.4f" % (rsamp.mean(), rsamp.std()))

# Do it with bootstrap

In [ ]:
data = A.rvs(size=100)

rsamp_bs = np.empty(1000)
for i in range(1000):
    ind = np.random.choice(range(len(data)),len(data))
    sample = data[ind,:]
    rsamp_bs[i],_ = pearsonr(sample[:,0],sample[:,1])

print("%.4f +/- %.4f" % (rsamp_bs.mean(), rsamp_bs.std()))

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(rsamp,alpha=0.5,label='extra samples')
plt.hist(rsamp_bs,alpha=0.5,label='bootstrap')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(linestyle='--')
plt.legend(fontsize=14)

# Bagging

In [ ]:
from sklearn import datasets
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.metrics import accuracy_score

## Test/train split

In [ ]:
iris = datasets.load_iris()
X = iris.data[50:,[1,2]]
y = iris.target[50:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
ntest = len(X_test)

In [ ]:
num_trees = 1000
ntrain = len(X_train)
all_acc = np.empty(num_trees)
y_pred = np.empty((num_trees,ntest))

trees = list()

for i in range(num_trees):
    
    # bootstrap sample the training data
    ind = np.random.choice(range(ntrain),ntrain)
    X_bs = X_train[ind,:]
    y_bs = y_train[ind]
    
    # train a new decision tree
    tree = DecisionTreeClassifier().fit(X_bs,y_bs)
    trees.append(tree)
    
    # test the tree
    y_pred[i,:] = tree.predict(X_test)
    all_acc[i] = accuracy_score(y_test,y_pred[i,:])

In [ ]:
plt.figure(figsize=(10,4))
plt.hist(all_acc)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(linestyle='--')
plt.yticks([])

In [ ]:
all_acc.mean()

In [ ]:
y_ensemble = scipy.stats.mode(y_pred).mode[0]

# Bagging in scikitlearn

In [ ]:
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier( DecisionTreeClassifier(), n_estimators=500)
model.fit(X_train, y_train)
accuracy_score( y_test, model.predict(X_test) )

# OOB

In [ ]:
model = BaggingClassifier( DecisionTreeClassifier(), n_estimators=500, oob_score=True)
model.fit(X_train, y_train)
model.oob_score_

In [ ]:
max_trees = 50
acc = np.empty(max_trees)
oob = np.empty(max_trees)
for ntrees in range(1,max_trees):
    model = BaggingClassifier( DecisionTreeClassifier(), n_estimators=ntrees, oob_score=True)
    model.fit(X_train, y_train)
    acc[ntrees] = accuracy_score( y_test, model.predict(X_test) )
    oob[ntrees] = model.oob_score_
    

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(max_trees),acc, linewidth=3, label='Test accuracy')
plt.plot(range(max_trees),oob, linewidth=3, label='OOB accuracy')
plt.xlim(0,50)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(linestyle='--')
plt.xlabel('Number of trees',fontsize=16)
plt.legend(fontsize=16, loc='lower right')